
# Databricks Coding Challenge - SQL
### Note: All questions should be done using SQL language

## Spark SQL and DataFrames 

In this section, you'll read in data to create a DataFrame in Spark.  We'll be reading in a dataset stored in the Databricks File System (DBFS).  Please see this [link](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html#databricks-file-system-dbfs) for more details on how to use DBFS.

##Understanding the data set 

###Overview:
The data set used throughout the coding assessment resembles telemetry data that any software as a service (SaaS) company might collect. One record represents the node hours for a single workload running on a transient cluster aggregated at the date and workload type level. This data set may be used to help Databricks understand consumption patterns and user behaviors on our platform. For example, we can inspect this data to understand if a given customer prefers our `automated` or `interactive` features, or understand which AWS instance types are preferred among all of our customers. 

###Format: 
 * JSON
 * Resides on S3

###Schema:
* date (String)
* nodeHours (Double)
* workloadType (String) (read more [here](https://databricks.com/product/aws-pricing#clusters))
* metadata (Struct)
 * clusterMetadata (Struct): Describes the cluster configuration
 * runtimeMetadata (Struct): Describes the software configuration
 * workloadMetadata (Struct): Describes the customer. Each shard may have one or many workspaces and each workspace may have zero or many clusters 



### Part A: SparkSQL and Dataframes 

In this section, you'll read in data to create a dataframe in Spark.  We'll be reading in a dataset stored in the Databricks File System (DBFS).  Please see this link for more details on how to use DBFS:
https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html#databricks-file-system-dbfs

Execute the command below to list the files in a directory that you will be analyzing.  There are several files in this test dataset.

In [0]:
%fs ls /databricks-coding-challenge/workloads

In [0]:
%fs head dbfs:/databricks-coding-challenge/workloads/part-00000-tid-7467717951814126607-30bac750-dd23-4160-a2a6-e57064ff0dc6-1506091-1-c000.json


### Question 1 (15 points):
Please create a temporary Spark SQL view called "workloads" from the json files in the directory listed up above

In [0]:
-- TOD0
-- Read external files directly using sql, temporary view is sesssion scoped we need re create the view if notebook session is restarted or detached. 
-- Both sql and spark supports reading files using glob pattern to search files. *.json will find all json files in the current directory. 
-- the external file listing are cached when it first runs use `REFRESH workloads`` to detect new files after the view was first ran
create or replace temporary view workloads
  as
select
  *
from
  JSON.`dbfs:/databricks-coding-challenge/workloads/*.json`


What is the schema for this table?

In [0]:
%python
## schema of the view. normally we use %sql describe table_name but it does not support for temporary views
spark.sql("select * from workloads").dtypes



### Question 2 (15 points):

Please print out all the unique workspaceId's for this dataset and order them such that workspaceId's are increasing in number.

In [0]:
-- TODO
-- Use . to access nested fields in the sturct column  
select distinct metadata.workloadMetadata.workspaceId as workspaceId
from workloads
order by workspaceId asc 




### Question 3 (15 points):

What is the number of unique clusters in this data set?  A cluster is identified by the `metadata.workloadMetadata.clusterId` field.

In [0]:
-- TODO
select
  count(distinct metadata.workloadMetadata.clusterId) as unique_cluster_count
from
  workloads

--- Total of 140,592 unique clusters

### Question 4 (15 points): 
What is the number of workload hours each day for the workspaceID - `-9014487477555684744`?

In [0]:
select
  date,
  sum(nodeHours) nodeHours
from
  workloads 
where metadata.workloadMetadata.workspaceId = -9014487477555684744
group by date
order by date asc

Databricks visualization. Run in Databricks to view.


Determine how many nodes are spot vs. on demand for a given cluster.

In [0]:
--- Does same cluster id spins up multiple times across dates or multiple times a day? answwer is Yes, as shown below  query highlights number of times each cluster id spins up across days.

select  metadata.workloadMetadata.clusterId, workloadtype, count(distinct date) NumUniqueDays
 from workloads
 group by metadata.workloadMetadata.clusterId, workloadtype
 having count(distinct date) > 1

In [0]:
-- TODO
-- Below query shows total number of spot vs On demand across all days
-- for given cluster id -1001061315011396571, there were 48 nodes were spot and 6 nodes were not, this cluster was spun up once. 
-- for given cluster id -1210922869916467973, there were 43 nodes were spot and 53 nodes were not, this cluster was spun up across more than one day. see the next cell query that breaks down by day. 
select  metadata.workloadMetadata.clusterId, 
      sum(case when metadata.clusterMetadata.containerIsSpot ="true" then 1 else 0 end)  as NumberOfSpotNodes,
      sum(case when metadata.clusterMetadata.containerIsSpot ="false" then 1 else 0 end)  as NumberOfOnDemandNodes,
      count(*) as  NumberOfNodes, 
      count(distinct date) as TotalDistnctDates
from workloads 
where metadata.workloadMetadata.clusterId in ( -1001061315011396571,-1210922869916467973)
group by  
      metadata.workloadMetadata.clusterId
order by  metadata.workloadMetadata.clusterId



In [0]:
-- for given cluster id -1210922869916467973, we can get a daily usage of spot vs on demand nodes since this cluster was spun up across many dates.
select
      date,   
      metadata.workloadMetadata.clusterId, 
      sum(case when metadata.clusterMetadata.containerIsSpot ="true" then 1 else 0 end)  as NumberOfSpotNodes,
      sum(case when metadata.clusterMetadata.containerIsSpot ="false" then 1 else 0 end)  as NumberOfOnDemandNodes,
      count(*) as  NumberOfNodes

from workloads 
where metadata.workloadMetadata.clusterId in (-1210922869916467973)
group by  
      date,  
      metadata.workloadMetadata.clusterId
order by  date, metadata.workloadMetadata.clusterId


### Question 5 (15 points): 

How many interactive node hours per day are there on the different Spark versions over time.

In [0]:
-- TODO
--- 4.2.x-scala2.11 is most used compared to other spark versions in interactive workload. 
select
  date,
  metadata.runtimeMetadata.sparkVersion,
  sum(nodehours) nodehours
from
  workloads
where workloadType = 'interactive'
group by
  date,
  metadata.runtimeMetadata.sparkVersion

Databricks visualization. Run in Databricks to view.

### Question 6 (25 points):
#### TPC-H Dataset
You're provided with a Line Items records from the TPC-H data set. The data is located in `/databricks-datasets/tpch/data-001/lineitem`.
Find the top two most recently shipped (shipDate) Line Items per Part using the simplest and most efficient approach.

You're free to use any combinations of SparkSQL, PySpark or Scala Spark to answer this challenge.

![](https://docs.deistercloud.com/mediaContent/Databases.30/TPCH%20Benchmark.90/media/tpch_schema.png?v=0)

In [0]:
%python
src ='/databricks-datasets/tpch/data-001/lineitem/lineitem.tbl'
schema =", ".join(['orderkey int', 'partkey int', 'suppkey int', 'lineNumber int', 'quantity int', 'extendedprice float', 'discount float', 'tax float', 'returnflag string', 'linestatus string', 'shipdate date', 'commitdate date', 'receiptdate date', 'shipinstruct string', 'shipmode string', 'comment string'])
tpc_h = (spark.read.format("csv") 
          .schema(schema)
          .option("header", False)
          .option("sep", "|")
          #.option("inferSchema", True)
          .load(src)
        )


In [0]:
%python
dbutils.fs.head('/databricks-datasets/tpch/data-001/lineitem/lineitem.tbl')

In [0]:
%python
display(tpc_h)

In [0]:
%python
tpc_h.createOrReplaceTempView('tpc_h')

Find the top two most recently shipped (shipDate) Line Items per Part using the simplest and most efficient approach.

- The most effiecent way to do this is to use row_number() and partition by part_key and order by most recent shipdate and filter the rownumber upto 2 to get the top 2.
- When we look at the spark execution plan most of the execution is driven by whostagecodegen except the exchange operator which is requred for sorting and cannot be avioded. 

In [0]:
--- The most effiecent way to do this is to use row_number() when we partition by part_key and order by most recent shipdate and filter the rownumber upto 2 to get the top 2.
--- When we look at the spark execution plan most of the execution is driven by whostagecodegen except the exchange operator which is requred for sorting and cannot be avioded.
with cte as (
  select
    shipdate,
    partkey,
    row_number() over(
      partition by partkey
      order by
        shipdate desc
    ) as rank_most_shipdate_by_partkey
  from
    tpc_h
)
select
  *
from
  cte
where
  rank_most_shipdate_by_partkey <= 2